# Preparation

## Authentication

Before requests to orchestration can be issued, we need to provide authentication details to the SDK. This can be done either via a configuration file or via the environment. Make sure to check out the [SAP generative AI hub SDK project description](https://pypi.org/project/generative-ai-hub-sdk/) for more details. Below you will find an example for authenticating via environment variables using this very notebook.

> **WARNING:**
> Below code should never be used in production scenarios and is only for the purpose of illustrating which environment variables to use!
> Credentials should never be defined in code!

In [5]:
import os

os.environ["AICORE_AUTH_URL"] = "https://saproxyshared.authentication.eu12.hana.ondemand.com/oauth/token"
os.environ["AICORE_BASE_URL"] = "redacted"
os.environ["AICORE_CLIENT_ID"] = "sb-na-7534ccc9-f8be-46e8-85e8-3e1a7afc2e08!b631918"
os.environ["AICORE_CLIENT_SECRET"] = "3c202202-3aad-46bc-b8b1-d3edf74b9407$IaIcyD5KfCsOFDlOEpL9pgOdkf5aSs-iqomIKdw7uoM="
os.environ["AICORE_ORCHESTRATION_DEPLOYMENT_URL"] = "https://aicore-proxy-ai180p104road.cfapps.eu12.hana.ondemand.com/v2/inference/deployments/d809af1946272325"
os.environ["AICORE_RESOURCE_GROUP"] = "default"

You will need to authenticate in every exercise.

# Content Filtering

Orchestration also supports content filtering to moderate input and output. For each, multiple filters can be applied that remove harmful content from texts. Depending on the filter, sensitivity of various types can be configured.

## Input Filter

Input filtering is handy for blocking out harmful content form e.g. user input. To make use of input filtering we first up need to define a basic pipeline again. We use a simple prompt template, that just passes text provided as a parameter to an LLM.

In [6]:
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue

llm = LLM(
    name="gemini-1.5-flash",
    version="latest",
    parameters={"max_tokens": 256, "temperature": 0.2},
)

template = Template(messages=[UserMessage("{{?text}}")])

Next up we will configure an AzureContentFilter using the corresponding SDK primitives.

In [7]:
from gen_ai_hub.orchestration.models.content_filter import AzureContentFilter

content_filter = AzureContentFilter(
    hate=0,
    sexual=0,
    self_harm=0,
    violence=0,
)

/var/folders/5t/n882phq131xgxldbsw6zbwwm0000gn/T/ipykernel_91979/652323566.py:3: DeprecationWarning: Importing AzureContentFilter from content_filter is deprecated. Please update your imports to use 'from gen_ai_hub.orchestration.models.azure_content_filter import AzureContentFilter'
  content_filter = AzureContentFilter(


**Note:** The lower the sensitivity value the higher the sensitivity. Zero (0) corresponds to the highest degree of content moderation. For further information please check out: https://learn.microsoft.com/en-us/azure/ai-services/content-safety/concepts/harm-categories?tabs=warning

Now that we have all modules defined, the only thing left to do is plugging everything together.

In [8]:
from gen_ai_hub.orchestration.service import OrchestrationService
from gen_ai_hub.orchestration.models.config import OrchestrationConfig

config = OrchestrationConfig(
    template=template,
    llm=llm,
    input_filters=[
        content_filter,
    ],
)

orchestration_service = OrchestrationService(
    api_url=os.environ["AICORE_ORCHESTRATION_DEPLOYMENT_URL"],
    config=config,
)

If the content filter detects a violation when performing an inference an error of type `OrchestrationError` will be raised.

In [9]:
from gen_ai_hub.orchestration.exceptions import OrchestrationError

try:
    result = orchestration_service.run(
        template_values=[
            TemplateValue(
                name="text",
                value="This is a hateful text!",
            ),
        ]
    )
except OrchestrationError as error:
    print(error.message)

Content filtered due to Safety violations. Please modify the prompt and try again.


The filter can be cleared by adjusting the prompt.

In [10]:
result = orchestration_service.run(
    template_values=[
        TemplateValue(
            name="text",
            value="This is a peaceful text!",
        )
    ]
)
print(result.orchestration_result.choices[0].message.content)

I agree!  It's great to hear that the text is peaceful.  

Could you tell me more about it?  What makes it peaceful?  Is it a story, a poem, a song, or something else?  I'd love to learn more about it. 😊 



## Output Filter

Similarly, also LLM output can be filtered. We will just use our already created filter and apply it to the LLM output within the configuration.

In [11]:
orchestration_service.config.output_filters = [content_filter]

Now let's try out if the filter works by prompting for an inherently violent lyrics from Johann Wolfgang von Goethe. Even though the version recited by the LLM might differ from the most common version, it should contain some form of suggested violence.

In [12]:
result = orchestration_service.run(
    template_values=[
        TemplateValue(
            name="text",
            value='Provide the lyrics of the ballad "Erlkönig" from Johann Wolfgang von Goethe in German and English.',
        )
    ]
)
print(result.orchestration_result.choices[0].message.content)

## Erlkönig

**German:**

Wer reitet so spät durch Nacht und Wind?
Es ist der Vater mit seinem Kind;
Er hat den Knaben wohl in dem Arm,
Er fasst ihn fest und hält ihn warm.

"Mein Sohn, was birgst du so bang dein Gesicht?"
"Vater, ich seh' den Erlköning,
Der grimme Mann im Nebel steht,
Er hat mir schöne Sachen versprochen."

"Mein Sohn, es ist ein Nebelstreich,
Das sind die Bäume, die sich biegen."
"Vater, ich seh' den Erlköning,
Der lockt mich mit seinem Zauberlied."

"Mein liebes Kind, komm, geh mit mir!
Spiel' schöne Spiele mit mir!
Manch bunte Blumen sind am Strand,
Meine Mutter hat goldene Kleider an."

"Vater, Vater, ich höre ihn singen,
Der Erlköning will mich gewinnen!"
"Mein Sohn, mein Sohn, ich seh' es wohl,
Es ist der Wind, der durch die Bäume rollt."

"Willst du mit mir, mein schönes Kind?
Du sollst bei uns im Garten spielen!
Meine Töchter haben Perlentracht,
Sie werden dich mit Freuden spielen."

"Vater, Vater, er fasst mich an!
Der Erlköning tut mir weh!"
"Mein Sohn, mein

**Note:** The behavior will differ from the input filter. Instead of raising an error, the returned content will be empty.

Feel free to try around with AzureContentFilter settings. If you set sensitivity for violence to 4 you will be able to read the lyrics of Erlkönig.

# Summary

In this exercise you learned how content filtering can be applied to input and output using orchestration. Now let's combine capabilities into a more complex scenario. Continue to [Exercise 3 - Orchestration Chatbot](./ex3.ipynb).